# Workbook
----------------------------------------

## Testing preprocess module
- 


In [1]:
%matplotlib nbagg

import preprocess

pp = preprocess.preprocess()
X, y = pp.getData("../homework2/data/data_small8.in", 50000) #path, clip length
# X, y = pp.getData("../serverTest/MusicClassification/data/data.in", 50000) #path, clip length

(X_center, y_center), (X_classifier, y_classifier), (X_test, y_test) = pp.splitData(
    X, y, randState=1985, TestTrainRatio=0.1, ClassifierCenterRatio=0.6)

print X_center.shape, X_classifier.shape, X_test.shape

X_center_features = pp.featureExtraction(X_center) #includes MFCC and normalization
print "X_center_features.shape", X_center_features.shape #Note: this just flatten everything


/Users/apple/anaconda/lib/python2.7/site-packages/IPython/kernel/__init__.py:13: ShimWarning: The `IPython.kernel` package has been deprecated. You should import from ipykernel or jupyter_client instead.
  "You should import from ipykernel or jupyter_client instead.", ShimWarning)
/Users/apple/anaconda/lib/python2.7/site-packages/librosa/core/audio.py:37: UserWarning: Could not import scikits.samplerate. Falling back to scipy.signal
  warnings.warn('Could not import scikits.samplerate. '


X size/number of songs: 80 Number of clips per song: 10 y size: 80
(28, 10, 50000) (44, 10, 50000) (8, 10, 50000)
--> After MFCC X.shape (28, 10, 12, 98)
--> X_train_flattened.shape (280, 12, 98)
--> After transpose X_train_flattened.shape (280, 98, 12)
X_center_features.shape (27440, 12)


## Find Center
- We need to play with K and maybe batch_size

In [20]:
from sklearn.cluster import MiniBatchKMeans
print "--------------", "START MiniBatchKMeans", "--------------"
n, dim = X_center_features.shape
clf = MiniBatchKMeans(n_clusters=1000, batch_size=int(n*0.1), max_iter=200).fit(X_center_features) #X : array-like, shape = [n_samples, n_features]
centroids = clf.cluster_centers_
print "centroids.shape", centroids.shape

-------------- START MiniBatchKMeans --------------
centroids.shape (1000, 12)


## VLAD

In [21]:
import vlad

trainX_centerSubstract, trainy_centerSubstract = vlad.my_vlad(centroids, X_classifier, y_classifier)
testX_centerSubstract, testy_centerSubstract = vlad.my_vlad(centroids, X_test, y_test)

print trainX_centerSubstract.shape

vlad before X.shape (44, 10, 50000)
vlad after X.shape (44, 10, 12, 98)
After transpose Xt.shape (44, 10, 98, 12)


KeyboardInterrupt: 

In [18]:
from sklearn.grid_search import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
import numpy as np
param_grid = {'n_neighbors': np.arange(3, 30),'weights':('uniform', 'distance'), 'algorithm':('auto','ball_tree', 'kd_tree', 'brute') }
np.set_printoptions(suppress=True)
grid_search = GridSearchCV(KNeighborsClassifier(), param_grid, verbose=3)
grid_search.fit(trainX_centerSubstract, trainy_centerSubstract)
print "----------------", "Done grid search", "----------------"

Fitting 3 folds for each of 216 candidates, totalling 648 fits
[CV] n_neighbors=3, weights=uniform, algorithm=auto ..................
[CV]  n_neighbors=3, weights=uniform, algorithm=auto, score=0.577181 -   0.0s
[CV] n_neighbors=3, weights=uniform, algorithm=auto ..................
[CV]  n_neighbors=3, weights=uniform, algorithm=auto, score=0.523810 -   0.0s
[CV] n_neighbors=3, weights=uniform, algorithm=auto ..................
[CV]  n_neighbors=3, weights=uniform, algorithm=auto, score=0.541667 -   0.0s
[CV] n_neighbors=3, weights=distance, algorithm=auto .................
[CV]  n_neighbors=3, weights=distance, algorithm=auto, score=0.590604 -   0.0s
[CV] n_neighbors=3, weights=distance, algorithm=auto .................
[CV]  n_neighbors=3, weights=distance, algorithm=auto, score=0.530612 -   0.0s
[CV] n_neighbors=3, weights=distance, algorithm=auto .................
[CV]  n_neighbors=3, weights=distance, algorithm=auto, score=0.541667 -   0.0s
[CV] n_neighbors=4, weights=uniform, alg

[Parallel(n_jobs=1)]: Done  31 tasks       | elapsed:    0.1s
[Parallel(n_jobs=1)]: Done 127 tasks       | elapsed:    0.6s



[CV] n_neighbors=23, weights=uniform, algorithm=auto .................
[CV]  n_neighbors=23, weights=uniform, algorithm=auto, score=0.476510 -   0.0s
[CV] n_neighbors=23, weights=uniform, algorithm=auto .................
[CV]  n_neighbors=23, weights=uniform, algorithm=auto, score=0.455782 -   0.0s
[CV] n_neighbors=23, weights=uniform, algorithm=auto .................
[CV]  n_neighbors=23, weights=uniform, algorithm=auto, score=0.562500 -   0.0s
[CV] n_neighbors=23, weights=distance, algorithm=auto ................
[CV]  n_neighbors=23, weights=distance, algorithm=auto, score=0.510067 -   0.0s
[CV] n_neighbors=23, weights=distance, algorithm=auto ................
[CV]  n_neighbors=23, weights=distance, algorithm=auto, score=0.489796 -   0.0s
[CV] n_neighbors=23, weights=distance, algorithm=auto ................
[CV]  n_neighbors=23, weights=distance, algorithm=auto, score=0.562500 -   0.0s
[CV] n_neighbors=24, weights=uniform, algorithm=auto .................
[CV]  n_neighbors=24, wei

[Parallel(n_jobs=1)]: Done 287 tasks       | elapsed:    1.3s
[Parallel(n_jobs=1)]: Done 511 tasks       | elapsed:    2.3s



[CV] n_neighbors=6, weights=uniform, algorithm=brute .................
[CV]  n_neighbors=6, weights=uniform, algorithm=brute, score=0.548611 -   0.0s
[CV] n_neighbors=6, weights=distance, algorithm=brute ................
[CV]  n_neighbors=6, weights=distance, algorithm=brute, score=0.577181 -   0.0s
[CV] n_neighbors=6, weights=distance, algorithm=brute ................
[CV]  n_neighbors=6, weights=distance, algorithm=brute, score=0.517007 -   0.0s
[CV] n_neighbors=6, weights=distance, algorithm=brute ................
[CV]  n_neighbors=6, weights=distance, algorithm=brute, score=0.569444 -   0.0s
[CV] n_neighbors=7, weights=uniform, algorithm=brute .................
[CV]  n_neighbors=7, weights=uniform, algorithm=brute, score=0.516779 -   0.0s
[CV] n_neighbors=7, weights=uniform, algorithm=brute .................
[CV]  n_neighbors=7, weights=uniform, algorithm=brute, score=0.503401 -   0.0s
[CV] n_neighbors=7, weights=uniform, algorithm=brute .................
[CV]  n_neighbors=7, weig

[Parallel(n_jobs=1)]: Done 648 out of 648 | elapsed:    2.9s finished


In [19]:
predict = grid_search.predict(testX_centerSubstract)
print grid_search.score(testX_centerSubstract, testy_centerSubstract)
print grid_search.best_params_

0.175
{'n_neighbors': 4, 'weights': 'distance', 'algorithm': 'auto'}
